<a href="https://colab.research.google.com/github/mrcruz117/linkedin-silverkite-forecast-templates/blob/main/LinkedIn_Silverkite_Parameter_Tuning_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Silverkite Parameter Tuning

In [69]:
#change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [70]:
# !pip install greykite

In [71]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from greykite.framework.templates.autogen.forecast_config import *
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.common.features.timeseries_features import *
from greykite.common.evaluation import EvaluationMetricEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from plotly.offline import iplot

In [72]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv')
future_df = pd.read_csv('future.csv')
future_df.head()
df.head()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2015,720.000885,0,0,0,3.68,41.305
1,1/2/2015,581.276773,0,0,0,4.73,131.574
2,1/3/2015,754.117039,0,0,0,7.23,162.700
3,1/4/2015,622.252774,0,0,0,10.96,160.281
4,1/5/2015,785.373319,0,0,0,6.92,51.077


In [73]:
# merge dfs

df = pd.concat([df, future_df])
df = df.reset_index(drop=True)
df.tail()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
2218,1/27/2021,NaN,0,0,0,3.33,39.664
2219,1/28/2021,NaN,0,0,0,1.67,195.314
2220,1/29/2021,NaN,0,0,0,-2.78,235.894
2221,1/30/2021,NaN,0,0,0,1.11,152.752
2222,1/31/2021,NaN,0,0,0,4.44,158.620


In [74]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,Date,y,Easter,Thanksgiving,Christmas,Temperature,Marketing


## Silverkite Prep

In [75]:
# time series name specifications
metadata = MetadataParam(
    time_col="Date",
    value_col="y",
    freq="D",
    train_end_date=pd.to_datetime("2020-12-31"),
)
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='Date', train_end_date=Timestamp('2020-12-31 00:00:00'), value_col='y')

In [76]:
# growth term possibilities
growth =  dict(
    growth_term=["linear", "quadratic", "sqrt"]
)
growth

{'growth_term': ['linear', 'quadratic', 'sqrt']}

In [77]:
# seasonalities
seasonality = dict(
    yearly_seasonality="auto",
    quarterly_seasonality="auto",
    monthly_seasonality="auto",
    weekly_seasonality="auto",
    daily_seasonality="auto",
)
seasonality

{'yearly_seasonality': 'auto',
 'quarterly_seasonality': 'auto',
 'monthly_seasonality': 'auto',
 'weekly_seasonality': 'auto',
 'daily_seasonality': 'auto'}

In [78]:
# holidays modeling
get_available_holiday_lookup_countries(["US"])
get_available_holidays_across_countries(countries=["US"],
                                        year_start=2015,
                                        year_end=2021)

['Christmas Day',
 'Christmas Day (Observed)',
 'Columbus Day',
 'Halloween',
 'Independence Day',
 'Independence Day (Observed)',
 'Juneteenth National Independence Day',
 'Juneteenth National Independence Day (Observed)',
 'Labor Day',
 'Martin Luther King Jr. Day',
 'Memorial Day',
 "New Year's Day",
 "New Year's Day (Observed)",
 'Thanksgiving',
 'Veterans Day',
 'Veterans Day (Observed)',
 "Washington's Birthday"]

In [79]:
# specifying events
events = dict(
    holidays_to_model_seperately = ["New Year's Day"],
    holiday_pre_num_days = 2,
    holiday_post_num_days = 2,
    holiday_pre_post_num_dict = {"New Year's Day":(3,1)},
    # adding one thats not there
    # daily_event_df_dict = {"elections": pd.DataFrame({
    #     "date": ["2016-11-03", "2020-11-03"],
    #     "event_name": ["elections"] * 2
    # })}
)
events

{'holidays_to_model_seperately': ["New Year's Day"],
 'holiday_pre_num_days': 2,
 'holiday_post_num_days': 2,
 'holiday_pre_post_num_dict': {"New Year's Day": (3, 1)}}

In [80]:
# changepoints: reflects changes in the trend
changepoints = dict(changepoints_dict=dict(method="auto"))

In [81]:
# regressors
regressors = dict(regressor_cols=["Easter", "Temperature", "Marketing"])
regressors

{'regressor_cols': ['Easter', 'Temperature', 'Marketing']}

In [82]:
# lagged regressors
lagged_regressors = dict(lagged_regressors_dict={"Temperature": "auto",
                                                 "Easter": "auto",
                                                 "Marketing": "auto"})

In [83]:
# autoregression: depoendent on the forecasting horizon
autoregression = dict(autoreg_dict= "auto")

### Model notes
- ridge regression model might be better.
- less risk of overfitting
- penalizes extreme slopes
  - makes more sense when we have less data.

- XGBoost
  - widely applicable
  - might work as well

In [84]:
# fitting algos

custom = dict(fit_algorithm_dict=[
    dict(fit_algorithm="linear"),
    dict(fit_algorithm="ridge"),
    dict(fit_algorithm="rf"),
    dict(fit_algorithm="gradient_boosting"),
])
custom

{'fit_algorithm_dict': [{'fit_algorithm': 'linear'},
  {'fit_algorithm': 'ridge'},
  {'fit_algorithm': 'rf'},
  {'fit_algorithm': 'gradient_boosting'}]}

## Building the Model

In [85]:
model_components = ModelComponentsParam(
    growth=growth,
    seasonality=seasonality,
    events=events,
    changepoints=changepoints,
    regressors=regressors,
    lagged_regressors=lagged_regressors,
    autoregression=autoregression,
    custom=custom
)

In [86]:
# cross-validation
evaluation_period = EvaluationPeriodParam(
    cv_min_train_periods=df.shape[0] - 180 - 31,
    cv_expanding_window=True,
    cv_max_splits=30,
    cv_periods_between_splits=16
)

In [87]:
# evaluation metric

evaluation_metric = EvaluationMetricParam(
    cv_selection_metric=EvaluationMetricEnum.RootMeanSquaredError.name
)

In [88]:
# model config
config = ForecastConfig(
    model_template=ModelTemplateEnum.SILVERKITE.name,
    forecast_horizon=31,
    model_components_param=model_components,
    evaluation_period_param=evaluation_period,
    evaluation_metric_param=evaluation_metric,
)

In [89]:
# forecasting
forecaster = Forecaster()
result = forecaster.run_forecast_config(df=df, config=config)

ValueError: ts column is not in input data